# Purpose:
- For **Fire Perimeter**:
    - Generate a circular polygon and crop the flight image based on that polygon
- For **Burn Severity**:
    - Crop the raster image based on input fire perimeter in GeoJSON (*product generated* or [CALFIRE](https://gis.data.ca.gov/datasets/CALFIRE-Forestry::california-fire-perimeters-all-1/explore)) or shapefile ([MTBS](https://www.mtbs.gov/viewer/index.html))
- For **either**:
    - Reproject a raster image to another coordinate reference systems (Check for burn severity raster images [MTBS](https://www.mtbs.gov/viewer/index.html))
    
***Due to GitHub file size limit, the post-RTC data is not available on this repository. Please following the steps in the 'radiometric_terrain_correction' repository to obtain the RTC data***

Cropped RTC image is available for the application of the other 3 notebooks in this repo.


In [1]:
import sys
from pathlib import Path

# Add the path to the utils folder to sys.path
utils_path = Path('../python').resolve()
sys.path.append(str(utils_path))

from pathlib import Path
from rasterio.crs import CRS
from crop_utils import (crop_image_by_coordinates, 
                        crop_image_by_geojson_shp, 
                        reproject_geotiff)

---
## Crop image by radius from center
Returns a cropped version of the input image based intersection of the specificed circle

**Parameters**:
- `path_to_images` (list): a list containing the paths to the images cropping from 
- `output_names` (list): a list containing the output names for the cropped images [**.tif** files]
- `center_longitude` (float): center longitude of the fire
- `center_latitude` (float): center latitude of the fire
- `radius_in_km` (float): radius from the coordinate in km to crop

In [2]:
data_dir = Path('/mnt/karenan/fasmee/fishlake/fishla_19601_23024_001_231017_L090_CX_01')
tifs = sorted(list(data_dir.rglob('./*_rtc.grd')))
tifs

[PosixPath('/mnt/karenan/fasmee/fishlake/fishla_19601_23024_001_231017_L090_CX_01/fishla_19601_23024_001_231017_HHHH_rtc.grd'),
 PosixPath('/mnt/karenan/fasmee/fishlake/fishla_19601_23024_001_231017_L090_CX_01/fishla_19601_23024_001_231017_HVHV_rtc.grd'),
 PosixPath('/mnt/karenan/fasmee/fishlake/fishla_19601_23024_001_231017_L090_CX_01/fishla_19601_23024_001_231017_VVVV_rtc.grd')]

In [3]:
path_to_images = tifs
center_longitude = -112.050134
center_latitude = 38.478988
radius_in_km = 5
output_names = [str(data_dir) + '/' + file.stem + '_cropped_' + str(radius_in_km) + 'km.tif' for file in path_to_images]
print(output_names)##

['/mnt/karenan/fasmee/fishlake/fishla_19601_23024_001_231017_L090_CX_01/fishla_19601_23024_001_231017_HHHH_rtc_cropped_5km.tif', '/mnt/karenan/fasmee/fishlake/fishla_19601_23024_001_231017_L090_CX_01/fishla_19601_23024_001_231017_HVHV_rtc_cropped_5km.tif', '/mnt/karenan/fasmee/fishlake/fishla_19601_23024_001_231017_L090_CX_01/fishla_19601_23024_001_231017_VVVV_rtc_cropped_5km.tif']


---
Now perform the cropping and the output will be saved at the working directory. Or you can edit `output_names` variable to set a location

In [4]:
for i in range(len(path_to_images)):
    crop_image_by_coordinates(path_to_images[i],
                              output_names[i], 
                              center_longitude, 
                              center_latitude, 
                              radius_in_km)

/mnt/karenan/fasmee/fishlake/fishla_19601_23024_001_231017_L090_CX_01/fishla_19601_23024_001_231017_HHHH_rtc_cropped_5km.tif is outputted!
/mnt/karenan/fasmee/fishlake/fishla_19601_23024_001_231017_L090_CX_01/fishla_19601_23024_001_231017_HVHV_rtc_cropped_5km.tif is outputted!
/mnt/karenan/fasmee/fishlake/fishla_19601_23024_001_231017_L090_CX_01/fishla_19601_23024_001_231017_VVVV_rtc_cropped_5km.tif is outputted!


In [5]:
##data_dir = Path('../data/latuna')
incs = sorted(list(data_dir.rglob('./*.inc')))
incs

[PosixPath('/mnt/karenan/fasmee/fishlake/fishla_19601_23024_001_231017_L090_CX_01/fishla_19601_23024_001_231017_L090_CX_01.inc')]

In [6]:
path_to_images = incs
##center_longitude = -118.2674
##center_latitude = 34.22957
##radius_in_km = 10
output_names = [str(data_dir) + '/' + file.stem + '_inc_' + str(radius_in_km) + 'km.tif' for file in path_to_images]
print(output_names)##

['/mnt/karenan/fasmee/fishlake/fishla_19601_23024_001_231017_L090_CX_01/fishla_19601_23024_001_231017_L090_CX_01_inc_5km.tif']


---
Now perform the cropping and the output will be saved at the working directory. Or you can edit `output_names` variable to set a location

In [7]:
for i in range(len(path_to_images)):
    crop_image_by_coordinates(path_to_images[i],
                              output_names[i], 
                              center_longitude, 
                              center_latitude, 
                              radius_in_km)

/mnt/karenan/fasmee/fishlake/fishla_19601_23024_001_231017_L090_CX_01/fishla_19601_23024_001_231017_L090_CX_01_inc_5km.tif is outputted!


---
## Crop image by a GeoJSON or Shapefile
Returns a cropped version of the input image corresponding to the polygon(s) in the input GeoJSON or Shapefile.

**Parameters**:
- `path_to_polygon_file` (str): path to the geojson or shapefile
- `path_to_images` (list): a list containing the paths to the images cropping from
- `output_names` (list): a list containing the output names for the cropped images. [**.tif** files]

The sample below will be cropping with a shapefile

In [3]:
shp_dir = Path('/mnt/karenan/fasmee/fasmee_fall_2023_burn_perimeter')
shps = sorted(list(shp_dir.glob('./*.shp')))
shps

[PosixPath('/mnt/karenan/fasmee/fasmee_fall_2023_burn_perimeter/fasmee_fall_2023_burn_perimeter.shp')]

In [12]:
##shps = '../data/bobcat/uavsar_perimeter/bobcat_perimeter_bilinear_inc_south.geojson'
shps = '/mnt/karenan/fasmee/fasmee_fall_2023_burn_perimeter/fasmee_fall_2023_burn_perimeter.shp'
shps

'/mnt/karenan/fasmee/fasmee_fall_2023_burn_perimeter/fasmee_fall_2023_burn_perimeter.shp'

In [13]:
data_dir = Path('/mnt/karenan/fasmee/fishlake/classify_fishla_01601_19601_230727_231017')
tifs = sorted(list(data_dir.glob('./*weighted*.tif')))
tifs

[PosixPath('/mnt/karenan/fasmee/fishlake/classify_fishla_01601_19601_230727_231017/fishlake_weighted_inc_merge_hv_0.tif'),
 PosixPath('/mnt/karenan/fasmee/fishlake/classify_fishla_01601_19601_230727_231017/fishlake_weighted_inc_merge_hv_1.tif')]

In [14]:
path_to_polygon_file = shps
path_to_images = tifs
output_names = [file.stem + '_perimeter_intersection_uavsar.tif' for file in tifs]
output_names

['fishlake_weighted_inc_merge_hv_0_perimeter_intersection_uavsar.tif',
 'fishlake_weighted_inc_merge_hv_1_perimeter_intersection_uavsar.tif']

---
Now perform the cropping and the output will be saved at the working directory. Or you can edit `output_names` variable to set a location

replace `path_to_polygon_file` with the geojson path if cropping by geojson

In [15]:
for i in range(len(path_to_images)):
    crop_image_by_geojson_shp(path_to_polygon_file,
                              path_to_images[i], 
                              output_names[i])

fishlake_weighted_inc_merge_hv_0_perimeter_intersection_uavsar.tif is outputted!
fishlake_weighted_inc_merge_hv_1_perimeter_intersection_uavsar.tif is outputted!


---
## Reproject Raster image to another coordinate reference systems
Used to reproject the MTBS burn severity map from ESRI:102039 to EPSG:4326, crs of UAVSAR data

The output could be used to extract the burn severity map of a particular fire or area, using `crop_image_by_coordinates`, `crop_image_by_geojson`, or `crop_image_by_shp` from above

**Parameters**
- `target_crs` (CRS): the crs to reproject the image to
- `input_raster_path` (str): path to the original raster image
- `output_raster_path` (str): path for the reprojected raster image [**.tif** files]

In [ ]:
data_dir = Path('../data/MTBS_sbs')
tifs = sorted(list(data_dir.glob('./*.tif')))
tifs

In [ ]:
target_crs = CRS.from_epsg(4326)   
input_raster_path = tifs[0]
output_raster_path = 'mtbs_CA_2009_crs4326.tif'

---
Now perform the cropping and the output will be saved at the working directory. Or you can edit `output_names` variable to set a location

In [ ]:
reproject_geotiff(target_crs, input_raster_path, output_raster_path)